The python node book is created to maintain all the loading jobs for the anm_real_graph. 
press run all to perform all the loading jobs. 
The description is givent for all the cells.

Importing everything neccassry.
bringing all the required data from the excel document provided. 

In [40]:
import pyTigerGraph as tg
import pandas as pd
import numpy as np
import datetime
conn = tg.TigerGraphConnection("http://172.16.16.124/", "anm_real_graph")
xlsx = pd.ExcelFile('data\Manifest data file.xlsx')
location_df = pd.read_excel(xlsx, 'Locations', na_values=['NA'])
boe_df = pd.read_excel(xlsx, 'Manifest header', na_values=['NA'])
event_df = pd.read_excel(xlsx, 'Stops', na_values=['NA'])


below are all the required functions we may need to execute the loading jobs

In [41]:
# function to change the type of the event to a list
def to_list(row):
    try:
        return row.split(",")
    except:
        return []


Creating a location dataframe.
Here we are creating a dataframe that is to be be pushed to tigergraph as location node.

In [42]:
location_df = location_df.loc[location_df['Locc#'].notnull()]
location_df.fillna('', inplace=True)
location_df['Zip'] = location_df['Zip'].apply(str)
location_df['Add1'] = location_df['Add1'].apply(str)
location_df['Add2'] = location_df['Add2'].apply(str)
location_df['Name'] = location_df['Name'].apply(str)
location_df['City'] = location_df['City'].apply(str)

location_df['full_addr'] = location_df.Name+","+location_df.Add1+"," + \
    location_df.Add2+","+location_df.City + "," + \
    location_df.State + " " + location_df.Zip

location_df.head()


,Locc#,Name,Add1,Add2,City,State,Country,Zip,full_addr
0,1,RRD CHICAGO - MFG,350 E CERMAK RD,,CHICAGO,IL,USA,606165507,"RRD CHICAGO - MFG,350 E CERMAK RD,,CHICAGO,IL ..."
1,2,LAKESIDE BOOK CRAWFORDSVILLE,1009 SLOAN ST,,CRAWFORDSVILLE,IN,USA,479332741,"LAKESIDE BOOK CRAWFORDSVILLE,1009 SLOAN ST,,CR..."
2,3,LAKESIDE BOOK WILLARD,1145 S CONWELL AVE,,WILLARD,OH,USA,448909392,"LAKESIDE BOOK WILLARD,1145 S CONWELL AVE,,WILL..."
3,4,LSC COMM US WARSAW,2801 W OLD ROAD 30,,WARSAW,IN,USA,465808783,"LSC COMM US WARSAW,2801 W OLD ROAD 30,,WARSAW,..."
4,5,RRD OLD SAYBROOK MFG-064RD,50 SCHOOL HOUSE RD,,OLD SAYBROOK,CT,USA,64754029,"RRD OLD SAYBROOK MFG-064RD,50 SCHOOL HOUSE RD,..."


Saving the DF as a csv for reference or furthur use.

In [43]:
location_df.to_csv('data/locations.csv', index=False)

Creating a bill of event dataframe.
Here we are creating a dataframe that is to be be pushed to tigergraph as bill_of_event node.

In [44]:
# boe_df.rename(columns={'Origin/Destination Lane':"Origin Destination Lane"})
boe_df['Name'] = boe_df['Carrier Name'] + boe_df['Origin Destination Lane']
boe_df['total_cost'] = boe_df['Cost Linehaul Cost'] + \
    boe_df['Cost STP Cost']+boe_df['Cost Other Accessorials']
boe_df.head()


,Manifest,Manifest Status,Ready to Book Date,Ready to Book Time,Carrier Code,Carrier Name,Carrier Type,Manifest Weight,Cost FS Cost,Cost Linehaul Cost,Cost STP Cost,Cost Other Accessorials,Origin Lane,Destination Lane,Origin Destination Lane,Name,total_cost
0,313325,C,2022-02-28,19:03:00,RDWYLTL,YRC YELLOW ROADWAY LTL,LTL,4175,111.52,743.51,0.0,0.0,MO650,RI028,OWENSVILLE- MO-CUMBERLAND- RI ...,YRC YELLOW ROADWAY LTLOWENSVILLE- MO-CUMBERLAN...,743.51
1,2951839,C,2022-03-01,09:47:00,UPGFLTL,TFORCE FREIGHT- INC.,LTL,795,41.16,140.00,0.0,0.0,RI028,NY110,CUMBERLAND- RI-INWOOD- NY ...,TFORCE FREIGHT- INC.CUMBERLAND- RI-INWOOD- NY ...,140.00
2,2952084,C,2022-03-01,09:47:00,UPGFLTL,TFORCE FREIGHT- INC.,LTL,1996,94.21,320.46,0.0,30.0,RI028,WI538,CUMBERLAND- RI-BOSCOBEL- WI ...,TFORCE FREIGHT- INC.CUMBERLAND- RI-BOSCOBEL- W...,350.46
3,2952085,C,2022-03-01,09:47:00,UPGFLTL,TFORCE FREIGHT- INC.,LTL,552,41.16,140.00,0.0,0.0,RI028,IL604,CUMBERLAND- RI-ROMEOVILLE- IL ...,TFORCE FREIGHT- INC.CUMBERLAND- RI-ROMEOVILLE-...,140.00
4,2953477,C,2022-03-01,21:24:00,UPGFLTL,TFORCE FREIGHT- INC.,LTL,5540,144.10,490.14,0.0,0.0,WI549,MD217,MENASHA- WI-HAGERSTOWN- MD ...,TFORCE FREIGHT- INC.MENASHA- WI-HAGERSTOWN- MD...,490.14


Creating a buisness entity dataframe.
Here we are creating a dataframe that is to be be pushed to tigergraph as buisness_entity node.

In [45]:
buisness_entity = boe_df[['Carrier Code',"Carrier Name","Carrier Type"]].drop_duplicates(subset = ['Carrier Code'])
buisness_entity['Carrier Type'] = buisness_entity['Carrier Type'].apply(to_list)
buisness_entity.head()

,Carrier Code,Carrier Name,Carrier Type
0,RDWYLTL,YRC YELLOW ROADWAY LTL,[LTL]
1,UPGFLTL,TFORCE FREIGHT- INC.,[LTL]
8,FXFELTL,MAIL FEDEX FREIGHT PRIORTY LTL,[LTL]
24,SAIALTL,MAIL SAIA MOTOR FREIGHT LTL,[LTL]
716,PGAO,PRESTIGE AUTO TRANSPORT,[CON]


Creating a event dataframe.
Here we are creating a dataframe that is to be be pushed to tigergraph as event node.

The event node is a very important node, so first thing we create is a unique id that is in the format of <bill_of_event-sequence-location>.
So by this ID you can infer which bill the event belongs to, whats the sequence of the event and where it happens, pretty cool!

In [46]:
event_df['Seq'] = event_df['Seq'].astype(str).apply(lambda x: x.replace('.0', ''))
event_df['Manifest'] = event_df['Manifest'].astype(str).apply(lambda x: x.replace('.0', ''))
event_df['Stop Location Code'] = event_df['Stop Location Code'].astype(str).apply(lambda x: x.replace('.0', ''))
event_df['id'] = event_df['Manifest']+"-" + event_df['Seq']+"-"+event_df['Stop Location Code']
event_df = event_df.rename(columns={
    "O=Origin,D=Final Stop": "type"
})
event_df.head()


,Manifest,Seq,type,Stop Location Code,Sch Date,Sch Time,Actual Date,Actual Time,id
0,313325,1,O,112301,20220228,1903,20220301,1903,313325-1-112301
1,313325,2,D,353062,20220303,715,20220303,715,313325-2-353062
2,2951839,1,O,353062,20220301,947,20220301,947,2951839-1-353062
3,2951839,2,D,344073,20220302,1025,20220304,1519,2951839-2-344073
4,2952084,1,O,353062,20220301,947,20220301,947,2952084-1-353062


We are gonna initialize all the datetime attributes, in the epoch format. 

In [47]:
df = event_df['Sch Date'].values
arr = []
for i in range(len(df)):
    a=df[i]
    try:
        val = str(int(a))
        date_obj = datetime.datetime.strptime(val ,'%Y%m%d')
        val = str(date_obj.date())
        arr.append(val)
    except:
        arr.append("NA")
event_df['epoch Sch Date'] = arr
df = event_df['Sch Time'].values
arr = []
for i in range(len(df)):
    a=df[i]
    try:
        val = int(a)
        h = str(int(val/100)).zfill(2)
        m = str(val%100).zfill(2)
        s = "00"
        val1 = h+":"+m+":"+s
        arr.append(val1)
    except:
        arr.append("NA")
event_df['epoch Sch Time'] = arr
df = event_df[['epoch Sch Date','epoch Sch Time']].values
arr = []
for i in range(len(df)):
    a=df[i]
    try:
        val = str(a)
        val1 = val[2:12]
        val2 = val[15:23]
        val=str(val1+" "+val2+",72")
        dt_obj = datetime.datetime.strptime(val, '%Y-%m-%d %H:%M:%S,%f')  
        epoch_msec = int(dt_obj.timestamp())
        arr.append(epoch_msec)
    except:
        arr.append("NA")
event_df['s_Epoch Time'] = arr
event_df = event_df[event_df['s_Epoch Time'] != 'NA']
event_df['s_Epoch_end'] = event_df['s_Epoch Time'].astype('int64') + 7200
df = event_df['Actual Date'].values
arr = []
for i in range(len(df)):
    a=df[i]
    try:
        val = str(int(a))
        date_obj = datetime.datetime.strptime(val ,'%Y%m%d')
        val = str(date_obj.date())
        arr.append(val)
    except:
        arr.append("NA")
event_df['epoch Actual Date'] = arr
df = event_df['Actual Time'].values
arr = []
for i in range(len(df)):
    a=df[i]
    try:
        val = int(a)
        h = str(int(val/100)).zfill(2)
        m = str(val%100).zfill(2)
        s = "00"
        val = h+":"+m+":"+s
        arr.append(val)
    except:
        arr.append("NA")
event_df['epoch Actual Time'] = arr
df = event_df[['epoch Actual Date','epoch Actual Time']].values
arr = []
for i in range(len(df)):
    a=df[i]
    try:
        val = str(a)
        val1 = val[2:12]
        val2 = val[15:23]
        val=str(val1+" "+val2+",72")
        dt_obj = datetime.datetime.strptime(val, '%Y-%m-%d %H:%M:%S,%f')  
        epoch_msec = int(dt_obj.timestamp())
        arr.append(epoch_msec)
    except:
        arr.append("NA")
event_df['a_Epoch Time'] = arr
event_df = event_df[event_df['a_Epoch Time'] != 'NA']
event_df['a_Epoch_end'] = event_df['a_Epoch Time'].astype('int64') + 7200
event_df['must_start'] = event_df['s_Epoch Time'].astype('int64') - 3600
event_df['must_end'] = event_df['a_Epoch_end'].astype('int64') +3600
event_df['load_time'] = 7200
event_df['type'] = event_df.type.apply(to_list)
event_df.head()

,Manifest,Seq,type,Stop Location Code,Sch Date,Sch Time,Actual Date,Actual Time,id,epoch Sch Date,epoch Sch Time,s_Epoch Time,s_Epoch_end,epoch Actual Date,epoch Actual Time,a_Epoch Time,a_Epoch_end,must_start,must_end,load_time
0,313325,1,[O],112301,20220228,1903,20220301,1903,313325-1-112301,2022-02-28,19:03:00,1646055180,1646062380,2022-03-01,19:03:00,1646141580,1646148780,1646051580,1646152380,7200
1,313325,2,[D],353062,20220303,715,20220303,715,313325-2-353062,2022-03-03,07:15:00,1646271900,1646279100,2022-03-03,07:15:00,1646271900,1646279100,1646268300,1646282700,7200
2,2951839,1,[O],353062,20220301,947,20220301,947,2951839-1-353062,2022-03-01,09:47:00,1646108220,1646115420,2022-03-01,09:47:00,1646108220,1646115420,1646104620,1646119020,7200
3,2951839,2,[D],344073,20220302,1025,20220304,1519,2951839-2-344073,2022-03-02,10:25:00,1646196900,1646204100,2022-03-04,15:19:00,1646387340,1646394540,1646193300,1646398140,7200
4,2952084,1,[O],353062,20220301,947,20220301,947,2952084-1-353062,2022-03-01,09:47:00,1646108220,1646115420,2022-03-01,09:47:00,1646108220,1646115420,1646104620,1646119020,7200


Now we are going to create a dataframe for the follow by edge.
First we group them by their bill, and then for each row we form a edge.

In [48]:
event_df.sort_values('id')
g_event_df = event_df.groupby('Manifest')
g_event_df.first()

,Seq,type,Stop Location Code,Sch Date,Sch Time,Actual Date,Actual Time,id,epoch Sch Date,epoch Sch Time,s_Epoch Time,s_Epoch_end,epoch Actual Date,epoch Actual Time,a_Epoch Time,a_Epoch_end,must_start,must_end,load_time
Manifest,,,,,,,,,,,,,,,,,,,
106248611,1,[O],352674,20220306,900,20220306,1304,106248611-1-352674,2022-03-06,09:00:00,1646537400,1646544600,2022-03-06,13:04:00,1646552040,1646559240,1646533800,1646562840,7200
106248612,1,[O],126728,20220313,800,20220314,800,106248612-1-126728,2022-03-13,08:00:00,1647138600,1647145800,2022-03-14,08:00:00,1647225000,1647232200,1647135000,1647235800,7200
106248613,1,[O],352674,20220320,400,20220321,1114,106248613-1-352674,2022-03-20,04:00:00,1647729000,1647736200,2022-03-21,11:14:00,1647841440,1647848640,1647725400,1647852240,7200
106248614,1,[O],352674,20220403,1200,20220403,1231,106248614-1-352674,2022-04-03,12:00:00,1648967400,1648974600,2022-04-03,12:31:00,1648969260,1648976460,1648963800,1648980060,7200
106281026,1,[O],369852,20220411,800,20220411,842,106281026-1-369852,2022-04-11,08:00:00,1649644200,1649651400,2022-04-11,08:42:00,1649646720,1649653920,1649640600,1649657520,7200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70004153,1,[O],363758,20220414,1400,20220414,1400,70004153-1-363758,2022-04-14,14:00:00,1649925000,1649932200,2022-04-14,14:00:00,1649925000,1649932200,1649921400,1649935800,7200
70004156,1,[O],361410,20220415,800,20220415,800,70004156-1-361410,2022-04-15,08:00:00,1649989800,1649997000,2022-04-15,08:00:00,1649989800,1649997000,1649986200,1650000600,7200
70004159,1,[O],129821,20220419,1400,20220419,1400,70004159-1-129821,2022-04-19,14:00:00,1650357000,1650364200,2022-04-19,14:00:00,1650357000,1650364200,1650353400,1650367800,7200


We set 'required' to true. 

In [59]:
source = []
target = []
for group in g_event_df.groups:
    temp_df = g_event_df.get_group(group)
    for i in range(len(temp_df)-1):
        source.append(temp_df['id'].iloc[i])
        target.append(temp_df['id'].iloc[i+1])

data = {
    "Source": source,
    "target": target,
    "required": True
}

followby_df = pd.DataFrame(data)

followby_df.head()


,Source,target,required
0,106248611-1-352674,106248611-2-189125,True
1,106248612-1-126728,106248612-2-352674,True
2,106248612-2-352674,106248612-3-189125,True
3,106248613-1-352674,106248613-2-189125,True
4,106248614-1-352674,106248614-2-189125,True


Save df for future reference as csv

In [50]:
followby_df.to_csv("data/follow_by.csv", index=False)

All the loading and mapping can be seen here. 

In [51]:
# location vertex
conn.upsertVertexDataFrame(location_df, 'location', 'Locc#', {
    "name": "Name",
    "address1": "Add1",
    "address2": "Add2",
    "city": "City",
    "zip": "Zip"
})

179310

In [52]:
# event vertex
conn.upsertVertexDataFrame(event_df,"event","id",attributes={
    "event_type":"type",
    "plan_start":"s_Epoch Time",
    "plan_end": "s_Epoch_end" ,
    "actual_start":"a_Epoch Time",
    "actual_end":"a_Epoch_end",
    "must_start_by":"must_start",
    "must_end_by" :"must_end",
    "load_time":"load_time"
})

71217

In [53]:
# bill_of_event vertex
conn.upsertVertexDataFrame(boe_df, "bill_of_events", "Manifest", {
    "name": "Name",
    "cost": "total_cost",
    "weight": "Manifest Weight",
})

29324

In [54]:
#  buisness_entity vertec
conn.upsertVertexDataFrame(buisness_entity,"buisness_entity","Carrier Code",{
    "name": "Carrier Name",
    "entity_type":"Carrier Type"
})

325

In [55]:
# belongs_to edge (event-(belongs_to)-bill_of_events) 
conn.upsertEdgeDataFrame(event_df, "event", "belongs_to", "bill_of_events", "id", "Manifest", {
    "belongs_to_sequence": "Seq"
})

71217

In [56]:
# happens_at edge (location-(happens_at)-event)
conn.upsertEdgeDataFrame(event_df, "location", "happens_at","event", "Stop Location Code", "id", {})

71217

In [57]:
# follow_by edge (event-(follow_by)-event)
conn.upsertEdgeDataFrame(followby_df, "event", "follow_by","event", "Source", "target", {
    "required":"required"
})

41893

In [58]:
# assigned_to edge (bill_of_events-(assigned_to)-buisness_entity)
conn.upsertEdgeDataFrame(event_df,"bill_of_events","assigned_to","buisness_entity","Manifest","Carrier Code",{})

29324